# A股量化交易系统 - 探索性分析

本 Notebook 演示如何使用量化交易系统进行数据获取、因子计算和策略回测。


In [ ]:
# 导入必要的库
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 导入项目模块
from src import (
    AkshareDataLoader,
    TechnicalFeatures,
    MACrossStrategy,
    RSIStrategy,
    BacktestEngine,
    load_config,
    setup_logging
)

# 配置日志
setup_logging()


## 1. 数据获取


In [ ]:
# 加载数据配置
data_config = load_config('../config/data_config.yaml')

# 初始化数据加载器
loader = AkshareDataLoader(data_config)

# 获取平安银行日线数据
symbol = "000001"
data = loader.fetch_daily_data(symbol, "2023-01-01", "2024-12-31")

print(f"数据形状: {data.shape}")
data.head()


In [ ]:
# 绘制价格走势图
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 收盘价
axes[0].plot(data.index, data['close'], label='收盘价', linewidth=1.5)
axes[0].set_ylabel('价格')
axes[0].set_title(f'{symbol} 价格走势')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 成交量
axes[1].bar(data.index, data['volume'], alpha=0.7, label='成交量')
axes[1].set_ylabel('成交量')
axes[1].set_xlabel('日期')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 2. 因子计算


In [ ]:
# 初始化技术指标计算器
features = TechnicalFeatures()

# 计算所有技术指标
data_with_features = features.calculate(data)

print(f"特征数量: {len(features.get_feature_names())}")
print(f"特征列表: {features.get_feature_names()}")

data_with_features.head()


In [ ]:
# 可视化技术指标
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# 价格和均线
axes[0].plot(data_with_features.index, data_with_features['close'], label='收盘价', linewidth=1.5)
axes[0].plot(data_with_features.index, data_with_features['sma_5'], label='SMA5', alpha=0.8)
axes[0].plot(data_with_features.index, data_with_features['sma_20'], label='SMA20', alpha=0.8)
axes[0].set_ylabel('价格')
axes[0].set_title('价格与均线')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(data_with_features.index, data_with_features['rsi_14'], label='RSI(14)', color='purple')
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.5)
axes[1].set_ylabel('RSI')
axes[1].set_title('RSI指标')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(data_with_features.index, data_with_features['macd'], label='MACD', color='blue')
axes[2].plot(data_with_features.index, data_with_features['macd_signal'], label='Signal', color='orange')
colors = ['g' if v >= 0 else 'r' for v in data_with_features['macd_hist']]
axes[2].bar(data_with_features.index, data_with_features['macd_hist'], color=colors, alpha=0.5)
axes[2].set_ylabel('MACD')
axes[2].set_title('MACD指标')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# 布林带
axes[3].plot(data_with_features.index, data_with_features['close'], label='收盘价', color='black', linewidth=1)
axes[3].plot(data_with_features.index, data_with_features['boll_upper'], label='上轨', color='red', alpha=0.7)
axes[3].plot(data_with_features.index, data_with_features['boll_middle'], label='中轨', color='blue', alpha=0.7)
axes[3].plot(data_with_features.index, data_with_features['boll_lower'], label='下轨', color='green', alpha=0.7)
axes[3].fill_between(data_with_features.index, data_with_features['boll_upper'], 
                      data_with_features['boll_lower'], alpha=0.1)
axes[3].set_ylabel('价格')
axes[3].set_xlabel('日期')
axes[3].set_title('布林带')
axes[3].legend(loc='upper left')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 3. 策略回测


In [ ]:
# 加载策略配置
strategy_config = load_config('../config/strategy_config.yaml')

# 初始化均线交叉策略
ma_strategy = MACrossStrategy(
    name="双均线策略",
    config={"short_window": 5, "long_window": 20}
)

# 初始化回测引擎
backtest_config = strategy_config.get('backtest', {})
engine = BacktestEngine(backtest_config)

# 执行回测
result = engine.run(ma_strategy, data_with_features)


In [ ]:
# 回测结果摘要
print("=" * 50)
print("回测结果摘要")
print("=" * 50)
print(f"总收益率:    {result.total_return:.2%}")
print(f"年化收益率:  {result.annual_return:.2%}")
print(f"夏普比率:    {result.sharpe_ratio:.2f}")
print(f"最大回撤:    {result.max_drawdown:.2%}")
print(f"胜率:        {result.win_rate:.2%}")
print(f"盈亏比:      {result.profit_factor:.2f}")
print(f"总交易次数:  {result.total_trades}")
print("=" * 50)


In [ ]:
# 绘制净值曲线
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 净值曲线
portfolio_values = result.portfolio_values
axes[0].plot(portfolio_values.index, portfolio_values, label='策略净值', linewidth=1.5)
axes[0].set_ylabel('净值')
axes[0].set_title('策略净值曲线')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 回撤曲线
peak = portfolio_values.expanding().max()
drawdown = (portfolio_values - peak) / peak * 100
axes[1].fill_between(drawdown.index, 0, drawdown, alpha=0.5, color='red', label='回撤')
axes[1].set_ylabel('回撤 (%)')
axes[1].set_xlabel('日期')
axes[1].set_title('回撤曲线')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 4. 策略对比


In [ ]:
# 初始化多个策略
strategies = [
    MACrossStrategy(name="MA(5,20)", config={"short_window": 5, "long_window": 20}),
    MACrossStrategy(name="MA(10,30)", config={"short_window": 10, "long_window": 30}),
    RSIStrategy(name="RSI(14)", config={"rsi_period": 14, "oversold": 30, "overbought": 70}),
]

# 策略对比
comparison = engine.compare_strategies(strategies, data_with_features)
comparison


In [ ]:
# 可视化策略对比
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 总收益率
axes[0].bar(comparison['strategy'], comparison['total_return'] * 100)
axes[0].set_ylabel('总收益率 (%)')
axes[0].set_title('总收益率对比')
axes[0].tick_params(axis='x', rotation=45)

# 夏普比率
axes[1].bar(comparison['strategy'], comparison['sharpe_ratio'])
axes[1].set_ylabel('夏普比率')
axes[1].set_title('夏普比率对比')
axes[1].tick_params(axis='x', rotation=45)

# 最大回撤
axes[2].bar(comparison['strategy'], comparison['max_drawdown'] * 100)
axes[2].set_ylabel('最大回撤 (%)')
axes[2].set_title('最大回撤对比')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 5. 参数优化


In [ ]:
# 均线策略参数优化
param_grid = {
    "short_window": [3, 5, 10, 15],
    "long_window": [20, 30, 40, 60]
}

optimization_result = engine.run_optimization(
    MACrossStrategy,
    data_with_features,
    param_grid
)

print("参数优化结果 (按夏普比率排序):")
optimization_result.head(10)


In [ ]:
# 参数优化热力图
pivot_table = optimization_result.pivot_table(
    values='sharpe_ratio', 
    index='short_window', 
    columns='long_window'
)

plt.figure(figsize=(10, 6))
plt.imshow(pivot_table.values, cmap='RdYlGn', aspect='auto')
plt.colorbar(label='夏普比率')
plt.xticks(range(len(pivot_table.columns)), pivot_table.columns)
plt.yticks(range(len(pivot_table.index)), pivot_table.index)
plt.xlabel('长期均线周期')
plt.ylabel('短期均线周期')
plt.title('参数优化热力图 (夏普比率)')

# 添加数值标注
for i in range(len(pivot_table.index)):
    for j in range(len(pivot_table.columns)):
        value = pivot_table.values[i, j]
        if not np.isnan(value):
            plt.text(j, i, f'{value:.2f}', ha='center', va='center', fontsize=10)

plt.tight_layout()
plt.show()


## 6. 下一步

- 尝试更多技术指标和因子
- 实现自定义策略
- 进行多股票回测
- 添加风险管理模块
